In [128]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [129]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [130]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [131]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [132]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [133]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import year, mean, format_number

result = df.filter("bedrooms=4").groupBy(year('date').alias('year')).agg(mean('price').alias('average_price'))
result = result.select('year', format_number('average_price', 2).alias('average_price'))
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|   296,363.88|
|2019|   300,263.70|
|2020|   298,353.78|
|2021|   301,819.44|
+----+-------------+



In [146]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

result = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3)).groupBy(year('date').alias('year')).agg(mean('price').alias('average_price'))
result = result.select('year', format_number('average_price', 2).alias('average_price'))
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|   292,725.69|
|2019|   287,287.82|
|2020|   294,204.16|
|2021|   294,211.46|
+----+-------------+



In [135]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

result = df.filter((df['bedrooms'] == 3) & (df['bathrooms'] == 3) & (df['floors'] == 2) & (df['sqft_lot'] >= 2000)).groupBy(year('date').alias('year')).agg(mean('price').alias('average_price'))
result = result.select('year', format_number('average_price', 2).alias('average_price'))
result.show()


+----+-------------+
|year|average_price|
+----+-------------+
|2022|   287,162.18|
|2019|   282,752.72|
|2020|   293,176.41|
|2021|   290,319.06|
+----+-------------+



In [136]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

filtered_df = df.filter(df['price'] >= 350000)

# Calculate the average price for each "view" rating
result = filtered_df.groupBy('view').agg(mean('price').alias('average_price'))

# Round "average_price" to two decimal places and format it as a string
result = result.select('view', format_number('average_price', 2).alias('average_price'))
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   7|   403,005.77|
|  51|   788,128.21|
|  15|   404,673.30|
|  54|   798,684.82|
|  11|   399,548.12|
|  29|   397,771.65|
|  69|   750,537.94|
|  42|   396,964.50|
|  87| 1,072,285.20|
|  73|   752,861.18|
|  64|   767,036.67|
|   3|   398,867.60|
|  30|   397,862.00|
|  34|   401,419.75|
|  59|   791,453.00|
|   8|   398,592.71|
|  28|   402,124.62|
|  22|   402,022.68|
|  85| 1,056,336.74|
|  16|   399,586.53|
+----+-------------+
only showing top 20 rows

--- 0.31671905517578125 seconds ---


In [137]:
# 7. Cache the the temporary table home_sales.
df.createOrReplaceTempView('home_sales')
spark.sql("cache table home_sales")

DataFrame[]

In [138]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [139]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

filtered_df = df.filter(df['price'] >= 350000)

# Calculate the average price for each "view" rating
result = filtered_df.groupBy('view').agg(mean('price').alias('average_price'))

# Round "average_price" to two decimal places and format it as a string
result = result.select('view', format_number('average_price', 2).alias('average_price'))
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|average_price|
+----+-------------+
|   7|   403,005.77|
|  51|   788,128.21|
|  15|   404,673.30|
|  54|   798,684.82|
|  11|   399,548.12|
|  29|   397,771.65|
|  69|   750,537.94|
|  42|   396,964.50|
|  87| 1,072,285.20|
|  73|   752,861.18|
|  64|   767,036.67|
|   3|   398,867.60|
|  30|   397,862.00|
|  34|   401,419.75|
|  59|   791,453.00|
|   8|   398,592.71|
|  28|   402,124.62|
|  22|   402,022.68|
|  85| 1,056,336.74|
|  16|   399,586.53|
+----+-------------+
only showing top 20 rows

--- 0.25815582275390625 seconds ---


In [140]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet('home_sales_partitioned')

In [141]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('home_sales_partitioned')
p_df

DataFrame[id: string, date: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string, date_built: int]

In [142]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_home_sales_p')

In [143]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

# Run the query to filter view ratings with an average price >= $350,000
filtered_df = p_df.filter(p_df['price'] >= 350000)
result = filtered_df.groupBy('view').agg(mean('price').alias('average_price'))

# Round "average_price" to two decimal places and format it as a string
result = result.select('view', format_number('average_price', 2).alias('average_price'))
result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
|   7|   403,005.77|
|  51|   788,128.21|
|  15|   404,673.30|
|  54|   798,684.82|
|  11|   399,548.12|
|  29|   397,771.65|
|  69|   750,537.94|
|  42|   396,964.50|
|  73|   752,861.18|
|  87| 1,072,285.20|
|  64|   767,036.67|
|   3|   398,867.60|
|  30|   397,862.00|
|  34|   401,419.75|
|  59|   791,453.00|
|   8|   398,592.71|
|  28|   402,124.62|
|  22|   402,022.68|
|  85| 1,056,336.74|
|  35|   401,934.21|
+----+-------------+
only showing top 20 rows

--- 0.34547877311706543 seconds ---


In [144]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [145]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False